# DNN Brasileirao Football Predict

### Import the pertinent libraries

In [1]:
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## Data

# http://www.football-data.co.uk/englandm.php Data

## Data Description

### Notes for Football Data

All data is in csv format, ready for use within standard spreadsheet applications. Please note that some abbreviations are no longer in use (in particular odds from specific bookmakers no longer used) and refer to data collected in earlier seasons. For a current list of what bookmakers are included in the dataset please visit http://www.football-data.co.uk/matches.php

#### Key to results data:

- Date = Match Date (dd/mm/yy)
- Home = Home Team
- Away = Away Team
- HG = Full Time Home Team Goals
- AG = Full Time Away Team Goals
- Res = Full Time Result (H=Home Win, D=Draw, A=Away Win)

##### Key to 1X2 (match) betting odds data:

- PH = Pinnacle home win odds
- PD = Pinnacle draw odds
- PA = Pinnacle away win odds

- MaxH = Oddsportal maximum home win odds
- MaxD = Oddsportal maximum draw win odds
- MaxA = Oddsportal maximum away win odds

- AvgH = Oddsportal average home win odds
- AvgD = Oddsportal average draw win odds
- AvgA = Oddsportal average away win odds


Current results (full time, half time)
Xcores - http://www.xcores.com

Bookmakers betting odds
Betbrain - http://www.betbrain.com
Oddsportal - http://www.oddsportal.com
Individual bookmakers

Betting odds for weekend games are collected Friday afternoons, and on Tuesday afternoons for midweek games.

In [2]:
df = pd.read_csv('../Data/Brazil_Serie_A/BRA.csv')

In [3]:
df.tail()

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA
2471,Brazil,Serie A,2018,23/08/18,01:00,Sport Recife,America MG,0.0,2.0,A,1.94,3.26,4.72,2.01,3.50,4.75,1.92,3.23,4.27
2472,Brazil,Serie A,2018,23/08/18,01:45,Fluminense,Corinthians,1.0,0.0,H,2.16,3.14,4.02,2.33,3.22,4.15,2.10,3.06,3.83
2473,Brazil,Serie A,2018,23/08/18,01:45,Gremio,Cruzeiro,1.0,1.0,D,1.88,3.10,5.51,1.94,3.20,5.51,1.86,3.08,4.89
2474,Brazil,Serie A,2018,23/08/18,23:30,Flamengo RJ,Vitoria,1.0,0.0,H,1.28,5.57,13.69,1.36,5.65,13.69,1.29,5.19,10.83
2475,Brazil,Serie A,2018,24/08/18,00:00,Atletico-MG,Vasco,0.0,0.0,D,1.48,4.47,7.57,1.50,4.54,8.50,1.46,4.26,6.93


## Separate the Features from the Labels
Features are the inputs for the neural net and labels are the ones to be predicted)

In [4]:
df.drop(['Country', 'League', 'Date', 'Time', 'HG', 'AG'], axis = 1, inplace = True)

In [5]:
df.dropna(inplace=True)

In [6]:
df['Res'].unique()

array(['D', 'H', 'A'], dtype=object)

In [7]:
df['Res'].replace(['H', 'D', 'A'],[1, 0, 2],inplace=True)

In [8]:
df['Res'].unique()

array([0, 1, 2], dtype=int64)

In [9]:
df['Season'] = df['Season'].apply(str)
df['Home'] = df['Home'].apply(str)
df['Away'] = df['Away'].apply(str)

In [10]:
df.dtypes

Season     object
Home       object
Away       object
Res         int64
PH        float64
PD        float64
PA        float64
MaxH      float64
MaxD      float64
MaxA      float64
AvgH      float64
AvgD      float64
AvgA      float64
dtype: object

In [11]:
x_data = df.drop(['Res'], axis=1)
y_label = df['Res']

## Extract the test sample from the train data
### test_size is the percentage of the data to being used for the tests (normally is 30%)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_label, test_size = 0.30)

## Create the feature columns
## Use "tf.feature_column.categorical_column_with_hash_bucket" para features em strings
   hash_bucket_size é o número de parâmetros possíveis (se a feature em questão for A, B ou C; hash_bucket_size=3)
## Use "tf.feature_column.categorical_column_with_identity" para formatos em int
   num_buckets é o número de parâmetros possíveis (se a feature for 1 até 7, num_buckets=7)

In [ ]:
df['Season'].nunique()

In [ ]:
season = tf.feature_column.categorical_column_with_hash_bucket('Season', hash_bucket_size = df['Season'].nunique())

home = tf.feature_column.categorical_column_with_hash_bucket('Home',hash_bucket_size = df['Home'].nunique())
away = tf.feature_column.categorical_column_with_hash_bucket('Away',hash_bucket_size = df['Away'].nunique())

In [ ]:
res = tf.feature_column.categorical_column_with_identity('Res', num_buckets = df['Res'].nunique())

## Passar as feature columns categoricas para embedded
## Usado para conseguir usar a Dense Neural Net

In [ ]:
df['Res'].nunique()

In [ ]:
season = tf.feature_column.embedding_column(season, dimension = df['Season'].nunique())

home = tf.feature_column.embedding_column(home, dimension = df['Home'].nunique())
away = tf.feature_column.embedding_column(away, dimension = df['Away'].nunique())

res = tf.feature_column.embedding_column(res, dimension = df['Res'].nunique())

## Criar feature columns numéricas

In [ ]:
X_train.columns

In [ ]:
#Odds
#hg = tf.feature_column.numeric_column('HG')
#ag = tf.feature_column.numeric_column('AG')

ph = tf.feature_column.numeric_column('PH')
pd = tf.feature_column.numeric_column('PD')
pa = tf.feature_column.numeric_column('PA')

max_h = tf.feature_column.numeric_column('MaxH')
max_d = tf.feature_column.numeric_column('MaxD')
max_a = tf.feature_column.numeric_column('MaxA')

avg_h = tf.feature_column.numeric_column('AvgH')
avg_d = tf.feature_column.numeric_column('AvgD')
avg_a = tf.feature_column.numeric_column('AvgA')

## Definir feature columns para input

In [ ]:
feat_cols = [season, home, away, ph, pd, pa, max_h, max_d, max_a, avg_h, avg_d, avg_a]

## Input para o model
   batch_size é o número de dados que serão usados em cada passo, shuffle definirá se os dados serão aleatórios em cada passo

In [ ]:
len(df)

In [ ]:
input_func = tf.estimator.inputs.pandas_input_fn(x = X_train, y = y_train, batch_size = 100, num_epochs = None, shuffle = True)

## Criação do modelo (O modelo salva automaticamente após ser treinado)
## CUIDADO COM O NUMERO DE NEURONIOS, TEM QUE SER IGUAL AO TREINAMENTO!!!!!
   Se o número de neurônios for alterado, delete o modelo e começe o treinamento de novo
   O número entre colchetes é o número de neurônios use uma camada apenas, com um número um pouco acima do número de inputs
   model_dir determina onde o modelo será salvo, se alguma característica da base de dados ou da rede neural for modificada, delete o modelo e reinicie o TREINAMENTO
    O modelo será salvo a partir da última rodada de treinamento

In [ ]:
model = tf.estimator.DNNClassifier([15, 12, 6, 3], feature_columns = feat_cols, n_classes = 3, model_dir ="./model")

## Treinar modelo
  steps é o número de passos (cada passo utiliza um "batch" ou pacote de dados)

In [ ]:
model.train(input_fn = input_func, steps = 50000)

## Evaluation do teste

In [ ]:
eval_func = tf.estimator.inputs.pandas_input_fn(x = X_test, y = y_test, batch_size = len(X_test), num_epochs = 1, shuffle = False)

## Teste
 O teste vai determinar a precisão do seu modelo

In [ ]:
results = model.evaluate(eval_func)

In [ ]:
results

## Input da previsão

In [ ]:
pred_func = tf.estimator.inputs.pandas_input_fn(x = X_test, batch_size = len(X_test), shuffle = False)

## Previsão
A previsão vai determinar outras características do resultado final do seu modelo

In [ ]:
prediction = model.predict(pred_func)
prediction = list(prediction)

final_preds = [pred['class_ids'][0] for pred in prediction]
print(final_preds)
print(classification_report(y_test, final_preds))

### Precision: o algoritmo acertou X das previsões (X/X_true)
### Recall: de 'X_true' parametros, o algoritmo acertou X (X/X_true)

batch 100
[15, 12, 6, 3]
steps 50000
             precision    recall  f1-score   support

          0       0.21      0.06      0.09       174
          1       0.58      0.80      0.67       397
          2       0.32      0.27      0.29       172

avg / total       0.43      0.50      0.45       743


+50k
             precision    recall  f1-score   support

          0       0.37      0.04      0.07       191
          1       0.53      0.92      0.67       362
          2       0.50      0.27      0.35       190

avg / total       0.48      0.52      0.44       743

+50k
             precision    recall  f1-score   support

          0       0.37      0.04      0.07       191
          1       0.53      0.92      0.67       362
          2       0.50      0.27      0.35       190

avg / total       0.48      0.52      0.44       743

+50k
             precision    recall  f1-score   support

          0       0.44      0.13      0.20       196
          1       0.55      0.86      0.67       365
          2       0.47      0.30      0.36       182

avg / total       0.50      0.53      0.47       743

In [51]:
import tensorflow as tf

In [52]:
from tensorflow import keras

In [53]:
dnn_keras_model = keras.Sequential()

In [54]:
dnn_keras_model.add(keras.layers.Dense(units=10, input_dim=10, activation='relu'))

In [55]:
dnn_keras_model.add(keras.layers.Dense(units=10, activation='relu'))
dnn_keras_model.add(keras.layers.Dense(units=10, activation='relu'))

In [56]:
dnn_keras_model.add(keras.layers.Dense(units=3, activation='softmax'))

In [57]:
from keras import losses,optimizers,metrics,activations

In [58]:
dnn_keras_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])


In [59]:
dnn_keras_model.compile(optimizer=tf.train.RMSPropOptimizer(0.01),
                        loss=keras.losses.categorical_crossentropy,
                        metrics=[keras.metrics.categorical_accuracy])

In [60]:
df_keras = pd.read_csv('../Data/Brazil_Serie_A/BRA.csv')

In [61]:
df_keras.tail()

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA
2471,Brazil,Serie A,2018,23/08/18,01:00,Sport Recife,America MG,0.0,2.0,A,1.94,3.26,4.72,2.01,3.50,4.75,1.92,3.23,4.27
2472,Brazil,Serie A,2018,23/08/18,01:45,Fluminense,Corinthians,1.0,0.0,H,2.16,3.14,4.02,2.33,3.22,4.15,2.10,3.06,3.83
2473,Brazil,Serie A,2018,23/08/18,01:45,Gremio,Cruzeiro,1.0,1.0,D,1.88,3.10,5.51,1.94,3.20,5.51,1.86,3.08,4.89
2474,Brazil,Serie A,2018,23/08/18,23:30,Flamengo RJ,Vitoria,1.0,0.0,H,1.28,5.57,13.69,1.36,5.65,13.69,1.29,5.19,10.83
2475,Brazil,Serie A,2018,24/08/18,00:00,Atletico-MG,Vasco,0.0,0.0,D,1.48,4.47,7.57,1.50,4.54,8.50,1.46,4.26,6.93


In [62]:
df_keras.dropna(inplace=True)

In [63]:
df_keras.drop(['Country', 'League', 'Date', 'Time', 'Home', 'Away'], axis = 1, inplace = True)

In [64]:
df_keras['Res'].replace(['H', 'D', 'A'],[1, 0, 2], inplace=True)

In [65]:
x_data_keras = df_keras.drop(['HG', 'AG', 'Res'], axis=1)

In [66]:
y_label_keras = df_keras[['HG', 'AG', 'Res']]

In [67]:
X_train_keras, X_test_keras, y_train_keras, y_test_keras = train_test_split(x_data_keras, y_label_keras, test_size = 0.30)

In [69]:
dnn_keras_model.fit(X_train_keras, y_train_keras, epochs=1000)

Epoch 1/1000
1732/1732 [==============================] - 3s 2ms/step - loss: 38.1202 - categorical_accuracy: 0.0982
Epoch 2/1000
1732/1732 [==============================] - 0s 92us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 3/1000
1732/1732 [==============================] - 0s 81us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 4/1000
1732/1732 [==============================] - 0s 80us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 5/1000
1732/1732 [==============================] - 0s 81us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 6/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 7/1000
1732/1732 [==============================] - 0s 80us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 8/1000
1732/1732 [==============================] - 0s 78us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 9/1000
1732/1732 [==============================] -

1732/1732 [==============================] - 0s 82us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 138/1000
1732/1732 [==============================] - 0s 77us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 139/1000
1732/1732 [==============================] - 0s 78us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 140/1000
1732/1732 [==============================] - 0s 77us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 141/1000
1732/1732 [==============================] - 0s 78us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 142/1000
1732/1732 [==============================] - 0s 78us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 143/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 144/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 145/1000
1732/1732 [=============================

1732/1732 [==============================] - 0s 82us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 274/1000
1732/1732 [==============================] - 0s 78us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 275/1000
1732/1732 [==============================] - 0s 78us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 276/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 277/1000
1732/1732 [==============================] - 0s 76us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 278/1000
1732/1732 [==============================] - 0s 82us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 279/1000
1732/1732 [==============================] - 0s 82us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 280/1000
1732/1732 [==============================] - 0s 83us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 281/1000
1732/1732 [=============================

1732/1732 [==============================] - 0s 82us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 410/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 411/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 412/1000
1732/1732 [==============================] - 0s 81us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 413/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 414/1000
1732/1732 [==============================] - 0s 78us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 415/1000
1732/1732 [==============================] - 0s 78us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 416/1000
1732/1732 [==============================] - 0s 82us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 417/1000
1732/1732 [=============================

1732/1732 [==============================] - 0s 78us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 546/1000
1732/1732 [==============================] - 0s 76us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 547/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 548/1000
1732/1732 [==============================] - 0s 78us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 549/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 550/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 551/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 552/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 553/1000
1732/1732 [=============================

1732/1732 [==============================] - 0s 82us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 682/1000
1732/1732 [==============================] - 0s 80us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 683/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 684/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 685/1000
1732/1732 [==============================] - 0s 77us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 686/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 687/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 688/1000
1732/1732 [==============================] - 0s 80us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 689/1000
1732/1732 [=============================

Epoch 749/1000
1732/1732 [==============================] - 0s 78us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 750/1000
1732/1732 [==============================] - 0s 78us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 751/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 752/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 753/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 754/1000
1732/1732 [==============================] - 0s 81us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 755/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 756/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 757/1000
1732/1732 [==============

1732/1732 [==============================] - 0s 81us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 886/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 887/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 888/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 889/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 890/1000
1732/1732 [==============================] - 0s 78us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 891/1000
1732/1732 [==============================] - 0s 79us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 892/1000
1732/1732 [==============================] - 0s 77us/step - loss: 38.3689 - categorical_accuracy: 0.0866
Epoch 893/1000
1732/1732 [=============================

In [70]:
predictions = dnn_keras_model.predict_classes(X_test_keras)

In [71]:
from sklearn.metrics import classification_report

In [94]:
y_test_keras.tail()

,HG,AG,Res
296,2,0,1
1200,2,2,0
1537,1,2,2
1701,3,1,1
416,0,1,2


In [92]:
y_test_keras['HG'] = y_test_keras['HG'].apply(int)
y_test_keras['AG'] = y_test_keras['AG'].apply(int)
y_test_keras['Res'] = y_test_keras['Res'].apply(int)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [93]:
print(classification_report(predictions, y_test_keras))

ValueError: Mix type of y not allowed, got types {'binary', 'multiclass-multioutput'}